## OpenAI Agent 的基础功能

使用 OpenAI Agent 的第一件事：关掉烦人的 `tracing`。

> 参考 [tracing](https://openai.github.io/openai-agents-python/tracing/) 文档：You can globally disable tracing by setting the env var `OPENAI_AGENTS_DISABLE_TRACING=1`

In [ ]:
import os
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "1"

In [1]:
# !pip install --upgrade openai

**启动 vllm 服务**

由于我们使用 vllm 启动 qwen3 推理服务，且希望它支持工具调用（Tool Calling），需要对 vllm 启动命令进行额外配置：

```bash
--enable-auto-tool-choice \
--tool-call-parser hermes \
```

参考：[Tool Calling](https://docs.vllm.ai/en/stable/features/tool_calling.html#qwen-models)

具体来说，你需要运行当前目录下的启动脚本：`bash vllm_server.sh`

In [21]:
from openai import AsyncOpenAI
from agents import (set_default_openai_client, set_default_openai_api,
                    Agent, Runner, ModelSettings)

**初始化 Agent**

使用 `Agent()` 初始化 Agent，使用 `Runner` 运行 Agent，`Runner` 有三种方法：

- 异步：Runner.run()
- 同步：Runner.run_sync()
- 流式（异步）；Runner.run_streamed()

In [3]:
# 创建自定义的 OpenAI 客户端
custom_client = AsyncOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-kcgyrk")
set_default_openai_client(custom_client)

# 使用 Chat Completions API
set_default_openai_api("chat_completions") 

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model="Qwen3-0.6B-FP8",
    model_settings=ModelSettings(
        temperature=0.6,
        top_p=0.95,
    ),
)

result = await Runner.run(agent, "介绍一下qwen3")
print(result.final_output.strip())

Qwen3是阿里巴巴集团研发的第三代大语言模型，由阿里巴巴集团研究院和阿里巴巴云共同开发，面向市场应用，属于大模型技术的前沿方向。以下是Qwen3的主要特点和优势：

1. **性能提升**  
   Qwen3在性能方面相比Qwen2有显著提升，支持更高的推理速度和更强的计算能力，能够处理更复杂、更庞大的数据集。

2. **应用场景广泛**  
   Qwen3适用于金融、医疗、教育、客服等多个领域，能够提供更精准的智能对话、文本生成和多轮推理服务。

3. **多模态支持**  
   Qwen3支持文本、图像、音频等多种输入形式，能够处理更丰富的交互场景，提升用户体验。

4. **持续优化**  
   Qwen3在研发过程中不断迭代，结合最新的技术，如大语言模型的优化、多模态处理能力提升，确保其持续进步。

Qwen3的推出标志着大模型技术在实际应用中的进一步成熟，为用户提供更高效、智能的服务解决方案。如果您有具体应用场景或想了解Qwen3在某个领域的具体表现，可以进一步提问。


**Function tools**

可以将任何 Python 函数用作工具。工具的名称是 Python 函数的名字；工具描述取自该函数的 docstring。

参考：[Tools](https://openai.github.io/openai-agents-python/tools/)

In [4]:
import json

from typing_extensions import TypedDict, Any
from agents import FunctionTool, RunContextWrapper, function_tool

根据城市名，返回该城市的天气。

```
{
  "Beijing": "Sunny",
  "Shanghai": "Cloudy",
  "Guangzhou": "Thunderstorm",
  "Chengdu": "Cloudy",
  "Hangzou": "Showery"
}
```

In [5]:
@function_tool  
async def fetch_weather(city: str) -> str:

    """Fetch the weather for a given city name.

    :param city: The city to fetch the weather for (e.g., "Beijing").
    :return: A string describing the weather (e.g., "Sunny", "Cloudy").
    """
    # we'd fetch the weather from a weather dict
    weather_dict = {
      "Beijing": "Sunny",
      "Shanghai": "Cloudy",
      "Guangzhou": "Thunderstorm",
      "Chengdu": "Cloudy",
      "Hangzou": "Showery"
    }
    return weather_dict.get(city, "Unknown")

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model="Qwen3-0.6B-FP8",
    model_settings=ModelSettings(
        temperature=0.6,
        top_p=0.9,
    ),
    tools=[fetch_weather],
)

In [6]:
citys = ['Beijing', 'Shanghai', 'Guangzhou', 'Chengdu', 'Hangzou', 'Shenzhen']

res = []
for city in citys:
    result = await Runner.run(agent, f"Tell me the weather in {city}.")
    res.append(result.final_output.strip())

In [7]:
print('\n'.join(res))

The weather in Beijing is sunny. Let me know if you need more details!
The weather in Shanghai is **Cloudy**. Let me know if you need more details! 🌤️
The weather in Guangzhou is currently a **thunderstorm**.
The weather in Chengdu is **Cloudy**.
The weather in Hangzou is Showery. Let me know if you need further details!
I don't have access to real-time weather data for Shenzhen. Could you please verify the city name again? If it's spelled correctly, I'll try to fetch it for you.


> 顺便一提，OpenAI 还提供了一些内置工具（Hosted tools），比如：
> 
> - `WebSearchTool`: 网络搜索
> - `CodeInterpreterTool`: 在沙盒环境执行代码
> - `HostedMCPTool`: 调用远程 MCP 服务
> - `LocalShellTool`: 在本地机器上运行 Shell 命令
> 
> 但是 `Chat Completions API` 模式下无法使用以上功能，我们当前正处于该模式。

OpenAI 提供两种风格的 API 接口：

- `OpenAIResponsesModel`: 使用最新的 [Responses API](https://platform.openai.com/docs/api-reference/responses)，即 `/v1/responses`
- `OpenAIChatCompletionsModel`: 使用旧的 [Chat Completions API](https://platform.openai.com/docs/api-reference/chat)，即 `/v1/chat/completions`

由于 vLLM 当前默认支持旧的 `Chat Completions API` 接口，而 OpenAI Agent 默认支持新接口，因此时常存在一些无法兼容的情况。参考：[models](https://openai.github.io/openai-agents-python/models/)

**MCP**

MCP 是一种为 LLM 提供工具和上下文的方法。OpenAI Agent 提供了三种方法连接 MCP 服务：

- `stdio`: 作为应用程序的子程序运行，可以将它视为“本地”运行
- `HTTP over SSE`: 远程运行 MCP Servers，可以通过 URL 连接它们
- `Streamable HTTP`: 使用 MCP 中定义的 Streamable HTTP 远程运行



参考：

- doc: [mcp](https://openai.github.io/openai-agents-python/mcp/)
- example: [filesystem_example/main.py](https://github.com/openai/openai-agents-python/blob/main/examples/mcp/filesystem_example/main.py)

下面我们用 **SSE** 的方法连接 MCP

首先启动 `FastMCP` 服务，在当前目录下打开 terminal：

```bash
cd sse_example
python server.py
```

如果成功运行，命令行输出如下：

```bash
$ python server.py 
INFO:     Started server process [27849]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8866 (Press CTRL+C to quit)
```

然后运行 Agent 作为客户端，调用刚刚启动的 `FastMCP` 服务：

```bash
python main.py
```

In [8]:
from agents.mcp import MCPServer, MCPServerSse

In [9]:
async def quick_test():
    async with MCPServerSse(
        name="SSE Python Server",
        params={"url": "http://localhost:8866/sse"},
    ) as server:
        custom_client = AsyncOpenAI(
            base_url="http://localhost:8000/v1",
            api_key="token-kcgyrk"
        )
        set_default_openai_client(custom_client)
        set_default_openai_api("chat_completions")

        agent = Agent(
            name="Assistant",
            instructions="Use the tools to answer the questions.",
            model="Qwen3-0.6B-FP8",
            mcp_servers=[server],
            model_settings=ModelSettings(
                temperature=0.6,
                top_p=0.9,
                tool_choice="required"
            ),
        )

        # 测试计算
        result = await Runner.run(starting_agent=agent, input="Add 17 and 6")
        print(f"Math result: {result.final_output}")

        # 测试秘密词
        result = await Runner.run(starting_agent=agent, input="What's the secret word?")
        print(f"Secret word: {result.final_output}")

await quick_test()

Math result: 

The result of adding 17 and 6 is 23.
Secret word: 

The secret word is **cherry**.


使用 `inspect` 获取类 `MCPServerStdio` 的构造函数的参数。

In [10]:
import inspect

sig = inspect.signature(MCPServerSse.__init__)

# 获取参数默认值
print("构造函数参数:")
for param_name, param in sig.parameters.items():
    if param_name != 'self':
        print(f"  - {param}")

print(f"\n源代码位置: {inspect.getfile(MCPServerSse)}")

构造函数参数:
  - params: 'MCPServerSseParams'
  - cache_tools_list: 'bool' = False
  - name: 'str | None' = None
  - client_session_timeout_seconds: 'float | None' = 5

源代码位置: /home/canva/miniconda3/lib/python3.12/site-packages/agents/mcp/server.py


**Handoffs**

Handoffs（交接）允许 Agent 将任务委托给另一个 Agent。这在不同 Agent 适用于不同领域的情况下特别有用。尤其对于流程类的服务特别有用，比如某个 Agent 处理退款程序，它先修改订单状态并执行退款，然后传递给下一个 Agent 给用户发送退款通知，并回复客户接下来的提问。

Agent 的 handoffs 参数，既可以接受 Agent，也可以接受自定义的 Handoff 对象。

参考：

- doc: [handoffs](https://openai.github.io/openai-agents-python/handoffs/)
- example: [message_filter.py](https://github.com/openai/openai-agents-python/blob/main/examples/handoffs/message_filter.py)

In [11]:
from __future__ import annotations

import json
import random

from openai import AsyncOpenAI
from agents import (Agent, HandoffInputData, Runner, function_tool, handoff, trace,
                    set_default_openai_client, set_default_openai_api, ModelSettings)
from agents.extensions import handoff_filters

In [12]:
@function_tool
def random_number_tool(max: int) -> int:
    """Return a random integer between 0 and the given maximum."""
    return random.randint(0, max)


def chinese_handoff_message_filter(handoff_message_data: HandoffInputData) -> HandoffInputData:
    # First, we'll remove any tool-related messages from the message history
    handoff_message_data = handoff_filters.remove_all_tools(handoff_message_data)

    # Second, we'll also remove the first two items from the history, just for demonstration
    history = (
        tuple(handoff_message_data.input_history[2:])
        if isinstance(handoff_message_data.input_history, tuple)
        else handoff_message_data.input_history
    )

    return HandoffInputData(
        input_history=history,
        pre_handoff_items=tuple(handoff_message_data.pre_handoff_items),
        new_items=tuple(handoff_message_data.new_items),
    )


# 创建自定义的 OpenAI 客户端
custom_client = AsyncOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-kcgyrk")
set_default_openai_client(custom_client)
set_default_openai_api("chat_completions")


first_agent = Agent(
    name="Assistant",
    model="Qwen3-0.6B-FP8",
    instructions="Be extremely concise.",
    tools=[random_number_tool],
)

chinese_agent = Agent(
    name="Chinese Assistant",
    model="Qwen3-0.6B-FP8",
    instructions="You only speak Chinese and are extremely concise.",
    handoff_description="A Chinese-speaking assistant.",
)

second_agent = Agent(
    name="Assistant",
    model="Qwen3-0.6B-FP8",
    instructions=(
        "Be a helpful assistant. If the user speaks Chinese, handoff to the Chinese assistant."
    ),
    handoffs=[handoff(chinese_agent, input_filter=chinese_handoff_message_filter)],
)

In [20]:
# 1. Send a regular message to the first agent
result = await Runner.run(first_agent, input="Hi, my name is Sora.")
print("[Step 1 done]")
print(result.final_output.strip())

# 2. Ask it to generate a number
result = await Runner.run(
    first_agent,
    input=result.to_input_list()
    + [{"content": "Can you generate a random number between 0 and 100?", "role": "user"}],
)
print("\n[Step 2 done]")
print(result.final_output.strip())

# 3. Call the second agent
result = await Runner.run(
    second_agent,
    input=result.to_input_list()
    + [
        {
            "content": "I live in New York City. Whats the population of the city?",
            "role": "user",
        }
    ],
)
print("\n[Step 3 done]")
print(result.final_output.strip())

# 4. Cause a handoff to occur
result = await Runner.run(
    second_agent,
    input=result.to_input_list()
    + [
        {
            "content": "我的名字是什么，我住在哪里？",
            "role": "user",
        }
    ],
)
print("\n[Step 4 done]")
print(result.final_output.strip())

[Step 1 done]
Hello, Sora! How can I assist you today?

[Step 2 done]
The random number between 0 and 100 is **78**. Let me know if you need anything else! 😊

[Step 3 done]
The population of New York City is approximately **8,440,000** as of the latest estimates. Let me know if you need help with anything else! 😊

[Step 4 done]
我的名字是 Sora，我住在美国纽约市。


**Central Agent**

在某些工作流中，你可能希望通过一个中心 Agent 编排网络，而非通过 Handoffs 移交控制权。

参考：[agents-as-tools](https://openai.github.io/openai-agents-python/tools/#agents-as-tools)

In [17]:
# 创建自定义的 OpenAI 客户端
custom_client = AsyncOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-kcgyrk")
set_default_openai_client(custom_client)
set_default_openai_api("chat_completions")

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
    model="Qwen3-0.6B-FP8",
)

chinese_agent = Agent(
    name="Chinese agent",
    instructions="You translate the user's message to Chinese",
    model="Qwen3-0.6B-FP8",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    model="Qwen3-0.6B-FP8",
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        chinese_agent.as_tool(
            tool_name="translate_to_chinese",
            tool_description="Translate the user's message to Chinese",
        ),
    ],
)

In [18]:
result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in Chinese.")
print(result.final_output.strip())

您好，怎么样吗？


**Guardrails**

Guardrails（护栏）是 Agent 的一种防护机制，用于检测用户输入，以防用户执行一些非常规操作，比如执行 100 次 Search API，这可能导致资损。

护栏有两种：
- 输入护栏 `Input guardrails`
- 输出护栏 `Output guardrails`

参考：[guardrails](https://openai.github.io/openai-agents-python/guardrails/)